In [212]:
# Updated RemoteVerifier matching current server (numeric coords, single connection)
import json, socket
from typing import Tuple
from ecpy.curves import Curve, Point

curve = Curve.get_curve("secp256k1")

class RemoteVerifier:
    def __init__(self, host: str = '127.0.0.1', port: int = 1337, timeout: int = 30):
        self.host = host
        self.port = port
        self.timeout = timeout
        self._sock: socket.socket | None = None
        self._A1: Point | None = None

    def _ensure_sock(self):
        if self._sock is None:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(self.timeout)
            s.connect((self.host, self.port))
            intro = s.recv(1 << 20).decode().strip()
            info = json.loads(intro)
            A1_x = int(info["A1_x"]) ; A1_y = int(info["A1_y"]) 
            self._A1 = Point(A1_x, A1_y, curve)
            self._sock = s

    def get_first_public_key(self) -> Point:
        self._ensure_sock()
        return self._A1  # type: ignore

    def verify_batched_proof_from_second_prover(self, A2: Point, proof: Tuple[Point, Point, int], msg: bytes) -> str:
        self._ensure_sock()
        R1, R2, s_val = proof
        payload = {
            "A2_x": str(int(A2.x)), "A2_y": str(int(A2.y)),
            "R1_x": str(int(R1.x)), "R1_y": str(int(R1.y)),
            "R2_x": str(int(R2.x)), "R2_y": str(int(R2.y)),
            "s": str(int(s_val)),
            "msg": msg.decode('latin-1'),
        }
        assert self._sock is not None
        self._sock.sendall((json.dumps(payload) + "\n").encode())
        resp = self._sock.recv(1 << 20).decode().strip()
        obj = json.loads(resp)
        print('[RemoteVerifier] Response:', obj)
        return obj.get('flag', 'Try again') if obj.get('ok') else 'Try again'

    def close(self):
        try:
            if self._sock is not None:
                self._sock.close()
        finally:
            self._sock = None
            self._A1 = None


In [213]:
pip install --quiet --no-input -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


## Schnorr batching with a shared Fiat–Shamir transcript

Schnorr is a signature scheme. For this exercise we are going to use an Elliptic Curve variant.

Points on an elliptic curve comprise a group. This means that there is a neutral element (zero in regular groups), there is an inverse for every element and the associativity law stands:


In [214]:
from ecpy.curves import Curve, Point

# instantiate the Curve
curve=Curve.get_curve("secp256k1")
G: Point = curve.generator
curve_order: int = curve.order

A = G * 5
B = G * 10
C = G * 20
point_at_infinity = G-G

# Neutral element (Point at Infinity)
assert (point_at_infinity + A) == A
# Inverse element
assert -(-A)==A
# Associativity
assert ((A+B) + C) == (A+ (B+C))


When Elliptic Curve is instantiated on top of a Finite Field (as is the case with the secp256k1 curve), the order of the group of points on an elliptic curve is also finite. We often use elliptic curves with a prime order (the order of a group is a prime number) in cryptography. What makes these curves useful is that the discrete logarithm problem on them is hard, so given two points `P` and `Q` on such a curve, it is computationally impossible to find such scalar `k` that `Q = k * P`. Scalar multiplication is described as summing the point `P` with itself `k` times.

We can use this property for various cryptographic tasks, such as creating signatures. In case of Schnorr Signatures the method is the following:

The signer samples a random scalar $a \in [1, curve\_order)$ then they compute a public key `A = G * a`, where `G` is the generator (a point on a curve that has been assigned as the generator of the group). They distribute this public key to whomever they want later to check their signature.

In the interactive version of the protocol, when the signer (who is also called a prover) wants to sign message `M`, they sample a new random scalar `k` called nonce and send a point `R = G * k` to the verifier.

The verifier then samples their own random scalar `e` and sends it back to the prover.

Next the prover computes `s = k + a * e` and sends to the verifier.

Lastly, the verifier computes `S_0 = G * s` and `S_1 = R + A * e` and checks that they are identical.


### Diagram: Interactive Schnorr protocol (single statement)

```
Prover (secret a, pub A = a·G)              Verifier
-----------------------------------------   ---------------------------
Choose k <-$ Z_q
R = k·G                           ----->     Receive R
                               <-----       Choose e <-$ Z_q
s = k + e·a  (mod q)             ----->     Receive s
                                            Check: s·G == R + e·A
```


In [215]:
import random
import hashlib

# Generate public key
a= random.randint(1, curve_order-1)
A = G * a

# Message
m = b"hello schnorr"

# Nonce
k = random.randint(1, curve_order-1)

# Random point
R = G * k

# Verifier's challenge
e= random.randint(1, curve_order-1)

# Prover's response
s = k + e * a

# Verifier's check
assert s * G == R + e * A


It would be very inconvenient to interact with the verifier directly every time we wanted to sign something. But there is a simple solution in public-coin protocols (protocols where everything the verifier generates immediately becomes public). It is called Fiat-Shamir Heuristic. The basic idea is that we need to hash all the previous values that were supposed to be submitted to the verifier with a secure hash. So instead of getting the verifier's challenge from them, we just compute `e= H(A || m || R)`
- Non-interactive (Fiat–Shamir) variant:
  - Both parties derive e = H(A || m || R) from the same transcript bytes
  - Prover sends (R, s) with s = k + e·a (mod q)
  - Verifier checks s·G == R + e·A


In [216]:
# Fiat–Shamir (non-interactive) Schnorr demo on secp256k1
from ecpy.curves import Curve, Point
import hashlib, random

curve = Curve.get_curve("secp256k1")
G: Point = curve.generator
q: int = curve.order

# Utilities

def i2b32(x: int) -> bytes:
    return int(x % curve.field).to_bytes(32, "big")


def hash_challenge(A: Point, m: bytes, R: Point) -> int:
    data = i2b32(A.x) + i2b32(A.y) + m + i2b32(R.x) + i2b32(R.y)
    return int.from_bytes(hashlib.sha256(data).digest(), "big") % q

# Keypair
a = random.randrange(1, q)
A = a * G

# Message
m = b"hello schnorr (fiat-shamir)"

# Nonce and commitment
k = random.randrange(1, q)
R = k * G

# Fiat–Shamir challenge and response
e = hash_challenge(A, m, R)
s = (k + e * a) % q

# Verify
lhs = s * G
rhs = R + e * A
print("Fiat–Shamir verify ok:", lhs == rhs)


Fiat–Shamir verify ok: True


You've just seen an example of a Non-Interactive Zero Knowledge Proof. Non-interactivity comes from the fact that the signature doesn't require interaction between the verifier and the prover, but why is it Zero Knowledge?

Well, think of it this way, in the interactive version of the protocol, if the Prover somehow manages to predict the challenge `e` that the Verifier sends them before they commit to `R`, they can break the whole protocol!

In [217]:
# Generate public key
a= random.randint(1, curve_order-1)
A = G * a

# Message
m = b"hello schnorr broken"

# Verifier's challenge (predicted by a malicious prover)
e= random.randint(1, curve_order-1)

# Nonce
k = random.randint(1, curve_order-1)

# Random point - A * e
R = G * k - A * e

# Prover's response
s = k

# Verifier's check
assert s * G == R + e * A

As you can see, the prover can now trivially break the protocol. They don't actually know the private key `a`. And since they didn't know the private key, there is no way that the verifier could learn the private key from this exchange, which is exactly what makes this Zero Knowledge.

Unfortunately, this kind of issue quite often appears in more complex protocols using Fiat-Shamir, when the protocol doesn't hash everything that is required. Let's look at an example of the protocol you will need to break. This time the verifier is checking a batched proof from 2 provers that decided to share their keys

In [218]:
# Generate two keypairs
a1 = random.randint(1, curve_order-1)
a2 = random.randint(1, curve_order-1)
A1 = G * a1
A2 = G * a2

# Message
msg = b"hello schnorr batching"

def serialize_point(point):
    return i2b32(point.x) + i2b32(point.y)
def deserialize_point(buf,curve):
    x = int.from_bytes(buf[:32], "big")
    y = int.from_bytes(buf[32:64], "big")
    return Point(x, y, curve)

def batched_fiat_shamir_signing(A1, A2, a1, a2, msg):
    # Generate first nonce
    k1 = random.randint(1, curve_order-1)
    R1 = k1 * G

    # Hash the first public key and R1 to generate a batching challenge
    data = serialize_point(A1) + serialize_point(R1)

    batch_challenge = int.from_bytes(hashlib.sha256(data).digest(), "big") % curve_order

    # Generate the second nonce
    k2 = random.randint(1, curve_order-1)
    R2 = k2 * G

    # Add the second public key, R2 and the message to the Fiat-Shamir transcript
    data += serialize_point(A2) + serialize_point(R2) + msg
    e = int.from_bytes(hashlib.sha256(data).digest(), "big") % curve_order

    # Generate the second response
    s = (k1 + e * a1 + batch_challenge * (k2 + e * a2)) % curve_order

    # Return the proof
    return (R1, R2, s)

def verify_batched_proof(proof, A1, A2, msg):
    R1, R2, s = proof

    # Hash the first public key and R1 to generate a batching challenge
    data = serialize_point(A1) + serialize_point(R1)

    batch_challenge = int.from_bytes(hashlib.sha256(data).digest(), "big") % curve_order

    # Add the second public key, R2 and the message to the Fiat-Shamir transcript
    data += serialize_point(A2) + serialize_point(R2) + msg
    e = int.from_bytes(hashlib.sha256(data).digest(), "big") % curve_order

    # Verify the proof
    lhs = s * G
    rhs = R1 + e * A1 + batch_challenge * (R2 + e * A2)
    return lhs == rhs

# Generate the proof
proof = batched_fiat_shamir_signing(A1, A2, a1, a2, msg)

# Verify the proof
assert verify_batched_proof(proof, A1, A2, msg)
    

 As you can see, we have batched two proofs together and saved some space (you don't need to send s1 and s2 now, just one s). However, there is a serious problem with batching, since the batched scalar does not depend on the second prover's parameters. This means that it is possible to forge the proof for public key A1 if you control the second prover. Implement the solution and get the flag from the server

In [219]:
message = b"Give us the flag"

class LocalVerifier:
    def __init__(self):
        self.A1 = G * random.randint(1, curve_order-1)
    
    def get_first_public_key(self):
        return self.A1
    
    def verify_batched_proof_from_second_prover(self, A2, proof, msg):
        if verify_batched_proof(proof, self.A1, A2, msg):
            return "Example flag"
        else:
            return "Try again"

    

local_verifier = LocalVerifier()

local_A1 = local_verifier.get_first_public_key()

def generate_malicious_proof(A1,msg):
    # Your code here
    A2 = G#your generated A2
    proof = (G,G,1)#your generated proof
    return A2,proof

(A2,proof) = generate_malicious_proof(local_A1, message)

print(local_verifier.verify_batched_proof_from_second_prover(A2, proof, message))



Try again


In [220]:
# RemoteVerifier with LocalVerifier-compatible API
import json, base64, socket
from typing import Dict, Any, Tuple
from ecpy.curves import Curve, Point

curve = Curve.get_curve("secp256k1")
FIELD = curve.field


class RemoteVerifier:
    """Remote verifier mirroring LocalVerifier API.

    Methods:
      - get_first_public_key() -> Point
      - verify_batched_proof_from_second_prover(A2, proof, msg) -> str
    """

    def __init__(self, host: str = "cryptotraining.zone", port: int = 1355, timeout: int = 30) -> None:
        self.host = host
        self.port = port
        self.timeout = timeout
        self._A1: Point | None = None
        self._sock: socket.socket | None = None

    def _connect(self):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(self.timeout)
        s.connect((self.host, self.port))
        self._sock = s

    def get_first_public_key(self) -> Point:
        self._connect()
        try:
            intro = self._sock.recv(1 << 20).decode().strip()
            info = json.loads(intro)
            A1_x = info.get("A1_x")
            A1_y = info.get("A1_y")
            if not A1_x or not A1_y:
                raise ValueError("Server did not provide A1_x or A1_y")
            self._A1 = Point(int(A1_x), int(A1_y), curve)
            print("[RemoteVerifier] Received A1 from server")
            return self._A1
        finally:
            try:
                s.close()
            except Exception:
                pass

    def verify_batched_proof_from_second_prover(self, A2: Point, proof: Tuple[Point, Point, int], msg: bytes) -> str:
        R1, R2, s_val = proof
        payload = {
            "A2_x": str(A2.x),
            "A2_y": str(A2.y),
            "R1_x": str(R1.x),
            "R1_y": str(R1.y),
            "R2_x": str(R2.x),
            "R2_y": str(R2.y),
            "s": str(int(s_val)),
            "msg": msg.decode('latin-1'),
        }
        try:
            assert self._sock is not None
            self._sock.sendall((json.dumps(payload) + "\n").encode())
            resp = self._sock.recv(1 << 20).decode().strip()
            obj = json.loads(resp)
            print("[RemoteVerifier] Server response:", obj)
            # Return the actual server flag if ok, else "Try again"
            return obj.get("flag", "Try again") if obj.get("ok") else "Try again"
        finally:
            try:
                s.close()
            except Exception:
                pass


In [221]:
remote_verifier = RemoteVerifier()

remote_A1 = remote_verifier.get_first_public_key()
(A2,proof) = generate_malicious_proof(remote_A1, message)

print(remote_verifier.verify_batched_proof_from_second_prover(A2, proof, message))


[RemoteVerifier] Received A1 from server
[RemoteVerifier] Server response: {'ok': False}
Try again
